## Embedding a Bokeh server in a Notebook

This notebook shows how a Bokeh server application can be embedded inside a Jupyter notebook. 
Code extracted from github:
https://github.com/bokeh/bokeh/blob/master/examples/howto/server_embed/notebook_embed.ipynb

IMPORTANT -serer embeding in Jupyter not working 100% - address of the Jupyer to be specified in case not in expected defualt position



In [ ]:
import yaml

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook

from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature
notebook_url="http://localhost:8890" 
output_notebook()

There are various application handlers that can be used to build up Bokeh documents. For example, there is a `ScriptHandler` that uses the code from a `.py` file to produce Bokeh documents. This is the handler that is used when we run `bokeh serve app.py`. Here we are going to use the lesser-known `FunctionHandler`, that gets configured with a plain Python function to build up a document. 

Here is the function `modify_doc(doc)` that defines our app:

In [ ]:
def modify_doc(doc):
    df = sea_surface_temperature.copy()
    source = ColumnDataSource(data=df)

    plot = figure(x_axis_type='datetime', y_range=(0, 25),
                  y_axis_label='Temperature (Celsius)',
                  title="Sea Surface Temperature at 43.18, -70.43")
    plot.line('time', 'temperature', source=source)

    def callback(attr, old, new):
        if new == 0:
            data = df
        else:
            data = df.rolling('{0}D'.format(new)).mean()
        source.data = ColumnDataSource(data=data).data

    slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")
    slider.on_change('value', callback)

    doc.add_root(column(slider, plot))

    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """))

Now we can display our application using ``show``, which will automatically create an ``Application`` that wraps ``modify_doc`` using ``FunctionHandler``. The end result is that the Bokeh server will call ``modify_doc`` to build new documents for every new sessions that is opened.

**Note**: If the current notebook is not displayed at the default URL, you must update the `notebook_url` parameter in the comment below to match, and pass it to `show`.

### Important = MI addition :
* code below is working only in case notebook_url="http://localhost:8888"
* otherwise notebook_url parameter has to be set correctly
* to find out the jupyter adress+port is not "easy" -
  * see discussion in https://github.com/bokeh/bokeh/issues/8096
  
* way aroud scan all jupyters instances urls  - this is the way how I made example working  


In [ ]:
show(modify_doc) # notebook_url="http://localhost:8888" 
# show(modify_doc, notebook_url="http://localhost:8888")

In [ ]:
from notebook import notebookapp
servers = list(notebookapp.list_running_servers())
for server in servers:
    print(server['url'])